In [ ]:
%env VECLIB_MAXIMUM_THREADS=1
%env MKL_NUM_THREADS=1
%env NUMEXPR_NUM_THREADS=1
%env OMP_NUM_THREADS=1

!make clean

import numpy as np
#np.show_config()

# Cache optimization

1. How cache works and its importance to performance
2. Stride analysis
3. Tiling

# Memory hierarchy

To show how cache works and its importance to performance, see table below:

| Type | Latency (cycles) |
| -- | -- |
| CPU registers | 0 |
| L1 cache | 4 |
| L2 cache | 10 |
| L3 cache | 50 |
| Main memory | 200 |
| Disk / drive | 100,000 |

(Excerpt from Figure 6.23 in CS:APP (3/e).)

Remarks:
* Disk (or SSD) is slow.  Don't do I/O unless absolutely necessary.
* CPU instruction is 100x faster than memory access.
* Cache hides the memory latency.

Nothing is fast without cache.

## How cache works

Assume we have a main memory of 64 bytes, and a cache of 8 bytes.

| Access # | Decimal addr | Binary addr | Hit or miss | Cache block (binary) |
| -- | -- | -- | -- | -- |
| 1 | 22 | 10110 | miss (cold) | 110 |
| 2 | 26 | 11010 | miss (cold) | 010 |
| 3 | 22 | 10110 | hit | 110 |
| 4 | 26 | 11010 | hit | 010 |
| 5 | 16 | 10000 | miss (cold) | 000 |
| 6 | 18 | 10010 | miss (conflict) | 010 |
| 7 | 26 | 11010 | miss (conflict) | 010 |

This is a direct-map cache.  To reduce conflict misses, we may use multi-way set associativity.  2-, 4-, or 8-way set associative cache is commonly used.  Full associativity is too expensive in circuit implementation.

# Cache block (line)

A cache block usually contains more than one byte or word.  In x86, the block (line) size is 64 bytes.  That is, when loading data from main memory to cache, it's done once a block, rather than byte by byte.

```cpp
// Sequential.
for (int i=0; i<nelem; ++i) { arr[i] = i; }
sw.lap();
for (int i=0; i<nelem; ++i) { arr[i] *= 3; }
elapsed = sw.lap();

// Skipping 2.
for (int i=0; i<nelem; ++i) { arr[i] = i; }
sw.lap();
for (int i=0; i<nelem; i+=2) { arr[i] *= 3; }
elapsed = sw.lap();

// ... 4, 8, 16, ... 1024.
```

In [ ]:
# Show how main memory (dram) access dominates runtime.
!make 01_skip_access ; ./01_skip_access

# Locality

While coding we usually don't have a lot of time to do detailed cache analysis.  Instead, we keep in mind that the code runs faster when it's more compact.  This is the concept of locality.

There are two kinds of locality: temporal and spatial.  Temporal locality means that a fixed address is reused in the near future.  Spatial locality means that the addresses close to the current address is reused in the near future.  The better locality, of either kind, improves the performance.  And the cache hierarchy is why locality works.

To take advantage of locality, programmers analyze by using "strides".  A stride is the number of indexes to elements to slide when accessing the data in arrays.  The most basic striding is sequential access, or the 1-stride.  Similarly, we may have n-strides.  The larger the stride is, the worse the locality.

Recall that x86 uses 64-byte cache blocks, and a double-precision floating point takes 8 bytes.

## Demonstrate by matrix-vector multiplication

In [ ]:
# Show how striding affects runtime.
!make 02_locality ; ./02_locality

While writing programs, it's much easier to know the stride than analyzing the cache behavior.  The latter, in many scenarios, is prohibitively difficult.

## Array majoring

Array majoring is directly related to locality.  The difference in the performance of matrix-vector multiplication is show for row- and column-majoring arrays.

In [ ]:
%%time
dim = 10000
float_rmajor = np.arange(dim*dim, dtype='float64').reshape((dim,dim))
float_cmajor = float_rmajor.T.copy().T
vec = np.arange(dim, dtype='float64')

In [ ]:
%%time
res_float_rmajor = np.dot(float_rmajor, vec)

In [ ]:
%%time
res_float_cmajor = np.dot(float_cmajor, vec)

### Integer matrix-vector multiplication

In [ ]:
%%time
dim = 10000
int_rmajor = np.arange(dim*dim, dtype='int64').reshape((dim,dim))
int_cmajor = int_rmajor.T.copy().T
vec = np.arange(dim, dtype='int64')

In [ ]:
%%time
res_int_rmajor = np.dot(int_rmajor, vec)

In [ ]:
%%time
res_int_cmajor = np.dot(int_cmajor, vec)

The performance difference of integer arrays is much larger than floating-point arrays.  Note that `double` and `int64` both take 8 bytes.  Reason: LAPACK / MKL / vecLib.

For the same reason, the floating-point multiplication is slightly faster than the integer.

# Tiling

This is a naive implementation of matrix-matrix multiplication:

```cpp
for (size_t i=0; i<mat1.nrow(); ++i)
{
    for (size_t k=0; k<mat2.ncol(); ++k)
    {
        double v = 0;
        for (size_t j=0; j<mat1.ncol(); ++j)
        {
            v += mat1(i,j) * mat2(j,k);
        }
        ret(i,k) = v;
    }
}
```

The matrices are row-major.  The stride for the second matrix is `ncol2`, so it doesn't have good locality.  This naive implementation is clear, but the performance is bad.

Matrix-matrix multiplication is one of the most important problems for numerical calculation, and there are many techniques available for making it fast.  Most if not all of them are about hiding the memory access latency.  Tiling is a basic technique that delivers impressive speed-up by reordering the calculation and making it cache-friendly.

In [ ]:
# Show how tiling improves runtime performance.
!make 03_matrix_matrix ; ./03_matrix_matrix

# Exercises

1. Consult the data sheet of one x86 CPU and one Arm CPU.  Make a table for the line size of each of the cache levels, and compare the difference between the two CPUs.
2. Write a program that uses tiling to speed up matrix-matrix multiplication, and do not require the matrix dimension to be multiples of the tile size. 

# References

* Gallery of Processor Cache Effects: http://igoro.com/archive/gallery-of-processor-cache-effects/
* Lecture Notes of Applications of Parallel Computers by David Bindel: https://www.cs.cornell.edu/~bindel/class/cs5220-s10/slides/lec03.pdf